In [84]:
#PigLab6NoteBook

! pip install pandas
! pip install openpyxl
!pip install rdflib
import pandas as pd
import rdflib
import hashlib
import numpy as np
import urllib.parse
from rdflib import Literal, Namespace, RDF, URIRef, Graph, RDFS, OWL
from rdflib.namespace import FOAF, XSD
from rdflib.plugins.sparql import prepareQuery
from pyspark.sql.functions import when, col, lit

# First data upload

In [85]:
# Define the file paths
files = ['Lab6/Copia de Labopat_mapa diagnostico_2021-2023.xlsx']

# Load the data into Pandas dataframes
dfs = []
for file in files:
    df = pd.read_excel(file, engine='openpyxl')
    dfs.append(df)

df = dfs[0]

df.head(20)

,Unnamed: 0,ID,Provicincia,Fecha,Patógeno,Resultado,Muestra,Edad/Granja,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,2021.0,50948/2021,Segovia,2021-01-08,SIV,Negativo,Pulmón,Cerdas,NaN,SIV,Virus de la Influenza Porcina,NaN,NaN,NaN
1,NaN,50974/2021,Segovia,2021-01-14,LI,Negativo,Intestino,Cebo,NaN,LI,Lawsonia intracelularis,NaN,NaN,=Ileitis proliferativa
2,NaN,50978/2021,Ávila,2021-01-14,LI,Negativo,Heces,Lechones 3 meses,NaN,MH,Mycoplasma hyopneumoniae,NaN,NaN,=neumonia enzootica porcina
3,NaN,51015/2021,Segovia,2021-01-15,LI,Negativo,Heces,4 meses,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,51061/2021,Murcia,2021-01-19,SIV,Negativo,Pulmón,Cebo,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,51075/2021,Murcia,2021-01-20,SIV,Negativo,Pulmón,-,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,51076/2021,Murcia,2021-01-20,SIV,Negativo,Pulmón,-,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,51076/2021,Murcia,2021-01-20,MH,Positivo,Pulmón,-,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,51108/2021,Segovia,2021-01-22,LI,Positivo,Heces,-,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,51132/2021,Burgos,2021-01-25,LI,Negativo,Heces,-,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
df = df.drop(['Unnamed: 0', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13'], 
             axis=1)

In [87]:
df.head(20)

,ID,Provicincia,Fecha,Patógeno,Resultado,Muestra,Edad/Granja
0,50948/2021,Segovia,2021-01-08,SIV,Negativo,Pulmón,Cerdas
1,50974/2021,Segovia,2021-01-14,LI,Negativo,Intestino,Cebo
2,50978/2021,Ávila,2021-01-14,LI,Negativo,Heces,Lechones 3 meses
3,51015/2021,Segovia,2021-01-15,LI,Negativo,Heces,4 meses
4,51061/2021,Murcia,2021-01-19,SIV,Negativo,Pulmón,Cebo
5,51075/2021,Murcia,2021-01-20,SIV,Negativo,Pulmón,-
6,51076/2021,Murcia,2021-01-20,SIV,Negativo,Pulmón,-
7,51076/2021,Murcia,2021-01-20,MH,Positivo,Pulmón,-
8,51108/2021,Segovia,2021-01-22,LI,Positivo,Heces,-
9,51132/2021,Burgos,2021-01-25,LI,Negativo,Heces,-


In [88]:
# Errors arised with the coding of the Age 
import unicodedata

def clean_text(text):
    if pd.isna(text):
        return None
    text = str(text).strip().lower()
    text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('utf-8')
    return text


# Create a column with the clean text
df['Edad/Granja'] = df['Edad/Granja'].apply(clean_text)
df['Provicincia'] = df['Provicincia'].apply(clean_text)
df['Resultado'] = df['Resultado'].apply(clean_text)

print(df['Provicincia'].unique())

['segovia' 'avila' 'murcia' 'burgos' 'pontevedra' 'badajoz' 'sevilla'
 'valladolid' 'toledo' 'asturias' 'granada' 'lugo' 'salamanca' 'zaragoza'
 'huelva' 'zamora' 'malaga' 'vitoria' 'valencia' 'a coruna' 'alava'
 'almeria' 'cuenca' 'ourense' 'leon' 'jaen' 'madrid' 'barcelona'
 'ciudad real' 'teruel']


In [89]:
#Rename Colums

df = df.rename(columns={
    'ID': 'SampleNumber',
    'Provicincia': 'Province',
    'Fecha': 'Date',
    'Patógeno': 'Pathogen',
    'Resultado': 'Result',
    'Muestra': 'SampleType',
    'Edad/Granja': 'Age', #Age of animal
})

# Set DiagnosticTest to "PCR" Country to "Spain", Breed to "Meat", and LabReference to "1".
df['DiagnosticTest'] = 'PCR'
df['Country'] = 'Spain'
df['Breed'] = 'Meat'
df["Lab_Reference"] = "6"






In [90]:
print(df['Result'].unique())

['negativo' 'positivo']


In [91]:
#Annotation Mapping

# Add Result mapping for NEG = 0 and POS =1
resultMapping = {
    "positivo": "1",
    "negativo": "0",
}
df["Result"] = df["Result"].map(resultMapping).fillna("Missing")

PathogenMapping = {

    "Lawsonia intracellularis": "Lawsonia intracellularis",
    "L. intracellularis": "Lawsonia intracellularis",
    "L.intracelullaris": "Lawsonia intracellularis",
    "M. hyopneumoniae": "Mycoplasma hyopneumoniae",
    "Myc. hyopneumoniae": "Mycoplasma hyopneumoniae",
    "Mycoplasma hyopneumoniae antistoffen" : "Mycoplasma hyopneumoniae",
    "Mycoplasma hyopneumoniae" : "Mycoplasma hyopneumoniae",
    "M. hyopneumoniae" : "Mycoplasma hyopneumoniae",
    "SIV" : "Swine influenza virus",
    "Influenza A" : "Swine influenza virus",
    "SIV" : "Swine influenza virus",
    "LI": "Lawsonia intracellularis",
    "MH": "Mycoplasma hyopneumoniae",
  
}

df["Pathogen"] = df["Pathogen"].map(PathogenMapping).fillna("Missing")

# SampleTypeMapping for the additional sample types
SampleTypeMapping = {
    'Pulmón': 'Tissue',
    'Intestino': 'Tissue',
    'Heces': 'Faeces',
    'Fluido Oral': 'Fluid',
    'Hisopo Rectal': 'Swab',
    'Hisopo Nasal': 'Swab',
    'Hisopo Laríngeo': 'Swab',
    'Raspados amígdalas': 'Swab',
    'Lechón': 'Tissue',
    'Calzas': 'Other',
    'Raspado bronquial': 'Swab',
    'Hisopo pulmonar': 'Swab',
    'Lavado Bronquial': 'BAL',
    'Raspado laríngeo': 'Swab',
    'Exudado nasal': 'Fluid',
    'Hisopo Traqueal': 'Swab',
    'Raspado Traqueal': 'Swab',
    'ESC NASAL': 'Swab',
    'RASP TRAQUEAL': 'Swab',
    'ESC LARINGEO': 'Swab',
    'LAV BRONCOALVEOLAR': 'BAL',
    'ESC PULMONAR': 'Swab',
    'SUERO': 'Blood',
    'LECHÓN': 'Tissue',
    'ABORTOS': 'Tissue',
    'PROCESADOS': 'Fluid',
    'LENGUAS': 'Tissue',
    'SEMEN': 'Fluid',
    'OTROS ÓRGANOS': 'Mixed',
    '19 MUESTRAS, PCR PRRS (2), PCR LEPTOSPIRAS (2)': 'Mixed',
    '2 MUESTRAS, PCR PRRS , PCR H. PARASUIS': 'Mixed',
    '1 SEMEN , PCR PRRS': 'Mixed',
    '1 MUESTRA PCR PRRS': 'Other',
    '4 MUESTRAS PCR PRRS (1), PCR PCV2(1)': 'Mixed',
    'MUESTRAS, PCR PCV2 (2), PCR PRRS(3)': 'Mixed',
    '3 MUESTRAS PCR PRRS (2)': 'Other',
    '4 MUESTRAS PCR PRRS': 'Other',
    '2 MUESTRAS PCR PRRS': 'Other',
    '8 MUESTRAS, PCR PRRS (3), PCR PCV2(2)': 'Mixed',
    'MUESTRAS, PCR PRRS (1) , PCR PCV2(1), PCR SALM(1)': 'Mixed',
    'HECES': 'Faeces',
    'INTESTINO': 'Tissue',
    'ESC RECTAL': 'Swab',

}

# Map the SampleType column using the defined mapping
df["SampleType"] = df["SampleType"].map(SampleTypeMapping).fillna("Missing")


# Province mapping
ProvinceMapping = {
    '-': 'Missing',
    'a coruna': 'ACoruna',
    'alava': 'Alava',
    'almeria': 'Almeria',
    'asturias': 'Asturias',
    'avila': 'Avila',
    'badajoz': 'Badajoz',
    'barcelona': 'Barcelona',
    'burgos': 'Burgos',
    'caceres': 'Caceres',
    'ciudad real': 'CiudadReal',
    'cuenca': 'Cuenca',
    'girona': 'Girona',
    'granada': 'Granada',
    'huelva': 'Huelva',
    'huesca': 'Huesca',
    'jaen': 'Jaen',
    'la rioja': 'LaRioja',
    'leon': 'Leon',
    'lugo': 'Lugo',
    'madrid': 'Madrid',
    'malaga': 'Malaga',
    'murcia': 'Murcia',
    'None': 'Missing',
    'ourense': 'Ourense',
    'pontevedra': 'Pontevedra',
    'salamanca': 'Salamanca',
    'segovia': 'Segovia',
    'sevilla': 'Sevilla',
    'tarragona': 'Tarragona',
    'teruel': 'Teruel',
    'toledo': 'Toledo',
    'valencia': 'Valencia',
    'valladolid': 'Valladolid',
    'vitoria': 'Alava',
    'zamora': 'Zamora',
    'zaragoza': 'Zaragoza',
}


df["Province"] = df["Province"].map(ProvinceMapping).fillna("Missing")




In [92]:
df

,SampleNumber,Province,Date,Pathogen,Result,SampleType,Age,DiagnosticTest,Country,Breed,Lab_Reference
0,50948/2021,Segovia,2021-01-08,Swine influenza virus,0,Tissue,cerdas,PCR,Spain,Meat,6
1,50974/2021,Segovia,2021-01-14,Lawsonia intracellularis,0,Tissue,cebo,PCR,Spain,Meat,6
2,50978/2021,Avila,2021-01-14,Lawsonia intracellularis,0,Faeces,lechones 3 meses,PCR,Spain,Meat,6
3,51015/2021,Segovia,2021-01-15,Lawsonia intracellularis,0,Faeces,4 meses,PCR,Spain,Meat,6
4,51061/2021,Murcia,2021-01-19,Swine influenza virus,0,Tissue,cebo,PCR,Spain,Meat,6
...,...,...,...,...,...,...,...,...,...,...,...
1231,67162/2023,Segovia,2023-12-21,Lawsonia intracellularis,1,Faeces,4 meses,PCR,Spain,Meat,6
1232,67187/2023,Segovia,2023-12-22,Lawsonia intracellularis,1,Swab,5 meses,PCR,Spain,Meat,6
1233,67205/2023,Segovia,2023-12-27,Lawsonia intracellularis,0,Faeces,-,PCR,Spain,Meat,6
1234,67209/2023,Segovia,2023-12-27,Swine influenza virus,0,Tissue,21 dias,PCR,Spain,Meat,6


In [93]:
print(df['Age'].unique())

['cerdas' 'cebo' 'lechones 3 meses' '4 meses' '-' 'lechones' '50 dias'
 '1 mes' '1-9 semanas' '6 semanas' '10 semanas' 'nuliparas' 'recria'
 '45 dias' '3 meses' 'reposicion' 'destete' 'transicion y cebo'
 'primerizas' 'cebo 100kg' '6 meses' 'transicion' '9 semanas' '7 dias'
 'cebo 130 dias' 'cerdas gestacion' '8 semanas' '80 dias' '180 dias'
 '10 meses' '12 semanas' '100 kg' 'final cebo' 'precebo' '90 dias'
 '140 dias' '3-9 semanas' '100 dias' 'primales' '5 meses' '110 dias'
 '18 dias' '35 kg' '2 meses' '3,5 meses' '5 semanas' '50-80 dias'
 'cebo 5 meses' '7 semanas' 'cerdas recria' '17 semanas' '8-9 meses'
 'cebo 70 kg' '26 semanas' '3 semanas' '14 semanas' '25 semanas' '60 dias'
 'lactacion' '75 dias' '2,5 meses' 'verraco' '35 dias' '30 dias'
 '16 semanas' '40 dias' '70 dias' 'cebo 30 kg' 'cerdas 10 meses'
 'post-destete' '9 meses' 'cerdas reposicion' '28 semanas' '8 meses'
 '18 semanas' '120 dias' 'cebo 120 dias' '49 dias' '11 semanas' 'primalas'
 '24 semanas' '21 dias' '1-3 anos' '

In [94]:
# PigAge mapping
PigStageMapping = {

    'cerdas': 'Sow',
'cebo': 'Finishing',
'lechones 3 meses': 'Finishing',
'4 meses': 'Finishing',
'-': 'Missing',
'lechones': 'Suckling',
'50 dias': 'Nursery',
'1 mes': 'Weaning',
'1-9 semanas': 'Mixed',
'6 semanas': 'Nursery',
'10 semanas': 'Nursery',
'nuliparas': 'Gilt',
'recria': 'Gilt',
'45 dias': 'Nursery',
'3 meses': 'Finishing',
'reposicion': 'Gilt',
'destete': 'Weaning',
'transicion y cebo': 'Mixed',
'primerizas': 'Sow',
'cebo 100kg': 'Finishing',
'6 meses': 'Finishing',
'transicion': 'Nursery',
'9 semanas': 'Nursery',
'7 dias': 'Suckling',
'cebo 130 dias': 'Finishing',
'cerdas gestacion': 'Sow',
'8 semanas': 'Nursery',
'80 dias': 'Finishing',
'180 dias': 'Finishing',
'10 meses': 'Finishing',
'12 semanas': 'Finishing',
'100 kg': 'Finishing',
'final cebo': 'Finishing',
'precebo': 'Nursery',
'90 dias': 'Finishing',
'140 dias': 'Finishing',
'3-9 semanas': 'Nursery',
'100 dias': 'Finishing',
'primales': 'Finishing',
'5 meses': 'Finishing',
'110 dias': 'Finishing',
'18 dias': 'Suckling',
'35 kg': 'Finishing',
'2 meses': 'Nursery',
'3,5 meses': 'Finishing',
'5 semanas': 'Nursery',
'50-80 dias': 'Nursery',
'cebo 5 meses': 'Finishing',
'7 semanas': 'Nursery',
'cerdas recria': 'Gilt',
'17 semanas': 'Finishing',
'8-9 meses': 'Finishing',
'cebo 70 kg': 'Finishing',
'26 semanas': 'Finishing',
'3 semanas': 'Nursery',
'14 semanas': 'Finishing',
'25 semanas': 'Finishing',
'60 dias': 'Nursery',
'lactacion': 'Suckling',
'75 dias': 'Finishing',
'2,5 meses': 'Finishing',
'verraco': 'Boar',
'35 dias': 'Nursery',
'30 dias': 'Nursery',
'16 semanas': 'Finishing',
'40 dias': 'Finishing',
'70 dias': 'Finishing',
'cebo 30 kg': 'Finishing',
'cerdas 10 meses': 'Finishing',
'post-destete': 'Nursery',
'9 meses': 'Finishing',
'cerdas reposicion': 'Gilt',
'28 semanas': 'Finishing',
'8 meses': 'Finishing',
'18 semanas': 'Finishing',
'120 dias': 'Finishing',
'cebo 120 dias': 'Finishing',
'49 dias': 'Nursery',
'11 semanas': 'Finishing',
'primalas': 'Gilt',
'24 semanas': 'Finishing',
'21 dias': 'Weaning',
'1-3 anos': 'Other',
'cebo 55 kg': 'Finishing',
'3 dias': 'Suckling',
'25 kg': 'Finishing',
'cebo 14 semanas': 'Finishing',
'90 kg': 'Finishing',
'80 kg': 'Finishing',
'54 dias': 'Nursery',
'30 kg': 'Finishing',
'20 kg': 'Finishing',
'160 dias': 'Finishing',
'14-16 semanas': 'Finishing',
'precebo 15 kg': 'Nursery',
'19 semanas': 'Finishing',
'4 semanas': 'Weaning',
'33 dias': 'Nursery',
'gestacion': 'Sow',
'13 semanas': 'Finishing',
'3-4 meses': 'Finishing',
'5-7 semanas': 'Nursery',
'50 kg': 'Finishing',
'4-9 semanas': 'Nursery',
'3-4 semanas': 'Weaning',
'ceno': 'Other',
'25-30 kg': 'Finishing',
'entrada cebo': 'Nursery',
'15 semanas': 'Finishing',
'10-13 semanas': 'Finishing',
'20 dias': 'Suckling',
'cebo 90 kg': 'Finishing',
'cebo 80 kg': 'Finishing',
'cerdas lactacion': 'Sow',
'cebo 40 kg': 'Finishing',
'85 dias': 'Finishing',
'final transicion': 'Mixed',
'20 semanas': 'Finishing',
'22 semanas': 'Finishing',
'2 a 7 meses': 'Finishing',
'3 a 5 meses': 'Finishing',
'42 dias': 'Nursery',
'2 anos': 'Other',
'150 dias': 'Finishing',

}

# Map the 'Edad' column using the English translation mapping
df['PigStage'] = df['Age'].map(PigStageMapping).fillna("Finishing")
df = df.drop(['Age'], axis=1)



In [95]:
df

,SampleNumber,Province,Date,Pathogen,Result,SampleType,DiagnosticTest,Country,Breed,Lab_Reference,PigStage
0,50948/2021,Segovia,2021-01-08,Swine influenza virus,0,Tissue,PCR,Spain,Meat,6,Sow
1,50974/2021,Segovia,2021-01-14,Lawsonia intracellularis,0,Tissue,PCR,Spain,Meat,6,Finishing
2,50978/2021,Avila,2021-01-14,Lawsonia intracellularis,0,Faeces,PCR,Spain,Meat,6,Finishing
3,51015/2021,Segovia,2021-01-15,Lawsonia intracellularis,0,Faeces,PCR,Spain,Meat,6,Finishing
4,51061/2021,Murcia,2021-01-19,Swine influenza virus,0,Tissue,PCR,Spain,Meat,6,Finishing
...,...,...,...,...,...,...,...,...,...,...,...
1231,67162/2023,Segovia,2023-12-21,Lawsonia intracellularis,1,Faeces,PCR,Spain,Meat,6,Finishing
1232,67187/2023,Segovia,2023-12-22,Lawsonia intracellularis,1,Swab,PCR,Spain,Meat,6,Finishing
1233,67205/2023,Segovia,2023-12-27,Lawsonia intracellularis,0,Faeces,PCR,Spain,Meat,6,Missing
1234,67209/2023,Segovia,2023-12-27,Swine influenza virus,0,Tissue,PCR,Spain,Meat,6,Weaning


In [96]:
df.to_csv('output/PigLab6_1.csv', sep=';', index=False)

## Second data upload

# Define the file paths
files1 = ['Lab6/Labopat_mapa diagnostico_2024_APP.csv',
        'Lab6/Labopat_mapa diagnostico_2024_BH.csv',
         'Lab6/Labopat_mapa diagnostico_2024_LI.csv',
         'Lab6/Labopat_mapa diagnostico_2024_MH.csv',
         'Lab6/Labopat_mapa diagnostico_2024_PED.csv',
         'Lab6/Labopat_mapa diagnostico_2024_PRRS.csv',
         'Lab6/Labopat_mapa diagnostico_2024_SI.csv']

# Load the data into Pandas dataframes
dfs1 = []
for file in files1:
    df1 = pd.read_csv(file, sep=";")
    dfs1.append(df1)


df1 = pd.concat(dfs1, ignore_index=True)


df1.head(20)

In [97]:
# Define the file paths
files1 = ['Lab6/Labopat_mapa diagnostico_2024_unificado.xlsx']

# Load the data into Pandas dataframes
dfs1 = []
for file in files1:
    df1 = pd.read_excel(file, engine='openpyxl')
    dfs1.append(df1)

df1 = dfs1[0]

df1.head(20)

,ID,Fecha,Provicincia,Patógeno,Resultado,Muestra,Edad/Granja
0,67337/2024,2024-01-10,Segovia,SIV,Negativo,PULMON,-
1,67360/2024,2024-01-11,Segovia,SIV,Negativo,PULMON,-
2,67440/2024,2024-01-17,Ourense,SIV,Negativo,PULMON,2 meses
3,67454/2024,2024-01-18,Badajoz,SIV,Negativo,PULMON,8 semanas
4,67515/2024,2024-01-19,Murcia,SIV,Negativo,FLUIDO ORAL,6 semanas
5,67516/2024,2024-01-19,Murcia,SIV,Positivo,FLUIDO ORAL,9 semanas
6,67608/2024,2024-01-24,Murcia,SIV,Positivo,FLUIDO ORAL,Entrada cebo
7,67614/2024,2024-01-24,Segovia,SIV,Negativo,PULMON,12 semanas
8,67641/2024,2024-01-25,Murcia,SIV,Positivo,FLUIDO ORAL,Cebo
9,67700/2024,2024-01-30,Segovia,SIV,Negativo,PULMON,90 días


In [98]:
#df1.to_csv('output/PigLab6_2_original.csv', sep=';', index=False)

In [99]:
#df1 = df1.drop(['Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Provincia'], axis=1)


#df1.head(20)

In [100]:
print(df1['Patógeno'].unique())

['SIV' 'MH' 'APP' 'PRRS' 'LI' 'B.HYO' 'PEDV']


In [101]:
# Errors arised with the coding of the Age 
import unicodedata

def clean_text(text):
    if pd.isna(text):
        return None
    text = str(text).strip().lower()
    text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('utf-8')
    return text


# Create a column with the clean text
df1['Edad/Granja'] = df1['Edad/Granja'].apply(clean_text)
df1['Provicincia'] = df1['Provicincia'].apply(clean_text)
df1['Resultado'] = df1['Resultado'].apply(clean_text)

print(df1['Provicincia'].unique())

['segovia' 'ourense' 'badajoz' 'murcia' 'valladolid' 'burgos' 'lugo' '-'
 'a coruna' 'jaen' 'malaga' 'tarragona' 'pontevedra' 'ciudad real'
 'barcelona' 'teruel' 'huesca' 'leon' 'girona' 'almeria' 'zaragoza'
 'toledo' 'salamanca' 'asturias' 'caceres' 'sevilla' 'zamora' 'valencia'
 'avila' 'tenerife' 'albacete' 'soria' 'cuenca' 'cordoba' 'huelva'
 'sin id' 'vic' 'la rioja']


In [102]:
#Rename Colums

df1 = df1.rename(columns={
    'ID': 'SampleNumber',
    'Provicincia': 'Province',
    'Fecha': 'Date',
    'Patógeno': 'Pathogen',
    'Resultado': 'Result',
    'Muestra': 'SampleType',
    'Edad/Granja': 'Age', #Age of animal
})

# Set DiagnosticTest to "PCR" Country to "Spain", Breed to "Meat", and LabReference to "1".
df1['DiagnosticTest'] = 'PCR'
df1['Country'] = 'Spain'
df1['Breed'] = 'Meat'
df1["Lab_Reference"] = "6"




In [103]:
print(df1['Result'].unique())

['negativo' 'positivo' None]


In [104]:
#Annotation Mapping

# Add Result mapping for NEG = 0 and POS =1
resultMapping = {
    "positivo": "1",
    "negativo": "0",
}
df1["Result"] = df1["Result"].map(resultMapping).fillna("Missing")

PathogenMapping = {

    "Lawsonia intracellularis": "Lawsonia intracellularis",
    "L. intracellularis": "Lawsonia intracellularis",
    "L.intracelullaris": "Lawsonia intracellularis",
    "M. hyopneumoniae": "Mycoplasma hyopneumoniae",
    "Myc. hyopneumoniae": "Mycoplasma hyopneumoniae",
    "Mycoplasma hyopneumoniae antistoffen" : "Mycoplasma hyopneumoniae",
    "Mycoplasma hyopneumoniae" : "Mycoplasma hyopneumoniae",
    "M. hyopneumoniae" : "Mycoplasma hyopneumoniae",
    "SIV" : "Swine influenza virus",
    "Influenza A" : "Swine influenza virus",
    "SIV" : "Swine influenza virus",
    "LI": "Lawsonia intracellularis",
    "MH": "Mycoplasma hyopneumoniae",
    'APP': "Actinobacillus pleuropneumoniae",
    'B.HYO': "Brachyspira hyodysenteriae",
    'PEDV': "Porcine epidemic diarrhea virus",
    'PRRS': "Porcine reproductive and respiratory syndrome virus"
        
  
}

df1["Pathogen"] = df1["Pathogen"].map(PathogenMapping).fillna("Missing")

# SampleTypeMapping for the additional sample types
SampleTypeMapping = {
    'Pulmón': 'Tissue',
    'Intestino': 'Tissue',           # Added based on the new dataset
    'Heces': 'Faeces',             
    'Fluido Oral': 'Fluid',         
    'Hisopo Rectal': 'Swab',         
    'Hisopo Nasal': 'Swab',         
    'Hisopo Laríngeo': 'Swab',      
    'Raspados amígdalas': 'Tissue',  
    'Lechón': 'Mixed',               
    'Calzas': 'Other',               
    'Raspado bronquial': 'Tissue', 
    'Hisopo pulmonar': 'Swab',       
    'Lavado Bronquial': 'Other',     
    'Raspado laríngeo': 'Tissue',    
    'Exudado nasal': 'Other',       
    'Hisopo Traqueal': 'Swab',      
    'Raspado Traqueal': 'Tissue',
    ###
    'ESC NASAL': 'Swab', 
    'RASP TRAQUEAL': 'Tissue',
    'ESC LARINGEO': 'Swab', 
    'LAV BRONCOALVEOLAR': 'BAL',
    'ESC PULMONAR': 'Swab', 
    'SUERO': 'Blood',
    'LECHÓN': 'Tissue',
    'ABORTOS': 'Tissue',
    'PROCESADOS': 'Fluid',
    'LENGUAS': 'Tissue',
    'SEMEN': 'Fluid',
    'OTROS ÓRGANOS': 'Mixed',
    '19 MUESTRAS, PCR PRRS (2), PCR LEPTOSPIRAS (2)': 'Mixed',
    '2 MUESTRAS, PCR PRRS , PCR H. PARASUIS': 'Mixed',
    '1 SEMEN , PCR PRRS': 'Mixed',
    '1 MUESTRA PCR PRRS': 'Other',
    '4 MUESTRAS PCR PRRS (1), PCR PCV2(1)': 'Mixed',
    'MUESTRAS, PCR PCV2 (2), PCR PRRS(3)': 'Mixed',
    '3 MUESTRAS PCR PRRS (2)': 'Other',
    '4 MUESTRAS PCR PRRS': 'Other',
    '2 MUESTRAS PCR PRRS': 'Other',
    '8 MUESTRAS, PCR PRRS (3), PCR PCV2(2)': 'Mixed',
    'MUESTRAS, PCR PRRS (1) , PCR PCV2(1), PCR SALM(1)': 'Mixed',
    'HECES': 'Faeces',
    'INTESTINO': 'Tissue',
    'ESC RECTAL': 'Swab', 
    'PULMON': 'Tissue',
'FLUIDO ORAL': 'Fluid',
'FLUIDOS ORAL': 'Fluid',
'ESC NASAL': 'Swab',
'RASP TRAQUEAL': 'Swab',
'ESC LARINGEO': 'Swab',
'LAV BRONCOALVEOLAR': 'BAL',
'ESC PULMONAR': 'Swab',
'SUERO': 'Blood',
'LECHÓN': 'Tissue',
'ABORTOS': 'Tissue',
'PROCESADOS': 'Fluid',
'LENGUAS': 'Tissue',
'SEMEN': 'Fluid',
'OTROS ÓRGANOS': 'Tissue',
'19 MUESTRAS, PCR PRRS (2), PCR LEPTOSPIRAS (2)': 'Mixed',
'2 MUESTRAS, PCR PRRS , PCR H. PARASUIS ': 'Mixed',
'1 SEMEN , PCR PRRS': 'Mixed',
'1 MUESTRA PCR PRRS': 'Other',
'4 MUESTRAS PCR PRRS (1), PCR PCV2(1)': 'Mixed',
'MUESTRAS, PCR PCV2 (2), PCR PRRS(3)': 'Mixed',
'3 MUESTRAS PCR PRRS (2)': 'Other',
'4 MUESTRAS PCR PRRS': 'Other',
'2 MUESTRAS PCR PRRS': 'Other',
'8 MUESTRAS, PCR PRRS (3), PCR PCV2(2)': 'Other',
'MUESTRAS, PCR PRRS (1) , PCR PCV2(1), PCR SALM(1)': 'Other',
'HECES': 'Faeces',
'INTESTINO': 'Tissue',
'ESC RECTAL': 'Swab',


}

# Map the SampleType column using the defined mapping
df1["SampleType"] = df1["SampleType"].map(SampleTypeMapping).fillna("Missing")


# Province mapping
ProvinceMapping = {
    '-': 'Missing',
'a coruna': 'ACoruna',
'alava': 'Alava',
'albacete': '',
'almeria': 'Almeria',
'asturias': 'Asturias',
'avila': 'Avila',
'badajoz': 'Badajoz',
'barcelona': 'Barcelona',
'burgos': 'Burgos',
'caceres': 'Caceres',
'ciudad real': 'CiudadReal',
'cordoba': '',
'cuenca': 'Cuenca',
'girona': 'Girona',
'granada': 'Granada',
'huelva': 'Huelva',
'huesca': 'Huesca',
'jaen': 'Jaen',
'la rioja': 'LaRioja',
'leon': 'Leon',
'lugo': 'Lugo',
'madrid': 'Madrid',
'malaga': 'Malaga',
'murcia': 'Murcia',
'None': 'Missing',
'ourense': 'Ourense',
'pontevedra': 'Pontevedra',
'salamanca': 'Salamanca',
'segovia': 'Segovia',
'sevilla': 'Sevilla',
'sin id': 'Missing',
'soria': 'Soria',
'tarragona': 'Tarragona',
'tenerife': 'SantaCruzDeTenerife',
'teruel': 'Teruel',
'toledo': 'Toledo',
'valencia': 'Valencia',
'valladolid': 'Valladolid',
'vic': 'Barcelona',
'vitoria': 'Alava',
'zamora': 'Zamora',
'zaragoza': 'Zaragoza',

}


df1["Province"] = df1["Province"].map(ProvinceMapping).fillna("Missing")




In [105]:
df1

,SampleNumber,Date,Province,Pathogen,Result,SampleType,Age,DiagnosticTest,Country,Breed,Lab_Reference
0,67337/2024,2024-01-10,Segovia,Swine influenza virus,0,Tissue,-,PCR,Spain,Meat,6
1,67360/2024,2024-01-11,Segovia,Swine influenza virus,0,Tissue,-,PCR,Spain,Meat,6
2,67440/2024,2024-01-17,Ourense,Swine influenza virus,0,Tissue,2 meses,PCR,Spain,Meat,6
3,67454/2024,2024-01-18,Badajoz,Swine influenza virus,0,Tissue,8 semanas,PCR,Spain,Meat,6
4,67515/2024,2024-01-19,Murcia,Swine influenza virus,0,Fluid,6 semanas,PCR,Spain,Meat,6
...,...,...,...,...,...,...,...,...,...,...,...
3050,70755/2024,2024-09-12,Burgos,Porcine epidemic diarrhea virus,0,Swab,cebo,PCR,Spain,Meat,6
3051,70996/2024,2024-10-01,Segovia,Porcine epidemic diarrhea virus,0,Faeces,cebo,PCR,Spain,Meat,6
3052,71012/2024,2024-10-02,Segovia,Porcine epidemic diarrhea virus,0,Faeces,40 dias,PCR,Spain,Meat,6
3053,71267/2024,2024-10-18,Ourense,Porcine epidemic diarrhea virus,0,Faeces,-,PCR,Spain,Meat,6


In [106]:
print(df1['Age'].unique())

['-' '2 meses' '8 semanas' '6 semanas' '9 semanas' 'entrada cebo'
 '12 semanas' 'cebo' '90 dias' 'futuras reproductoras' 'lechones'
 '4 semanas' '5 semanas' 'transicion' 'cerdas nuliparas' '71 dias'
 'recria' '70 dias' '14 semanas' '3 semanas' '25 dias' 'lactacion'
 'cerdas' '5-10 semanas' '4-10 semanas' '5-9 semanas' '4-8 semanas'
 'primalas' '7 semanas' '5-8 semanas' '240 dias' '5 meses' '2-3 semanas'
 '6-7 semanas' '10 semanas' '1-2 semanas post-destete' '100 dias'
 '6-8 semanas' 'cerdas gestacion' 'lechones transicion' 'reproductoras'
 '50 dias' '4 meses' 'reposicion' 'lechones destete' '80 dias' '3 meses'
 'lechonera' '42 dias' None '13 semanas' 'lechones 4 semanas'
 'lechones 5-6 semanas' 'lechones 15 dias' '7 meses' 'lechones 3 semanas'
 'lechones 6 semanas' 'lechones 54 dias' 'nacidos muertos' 'nuliparas'
 'lechones 10 semanas' 'lechones 56 dias' 'multiparas'
 'lechones 1-8 semanas' '25 semanas' 'lechones lactacion'
 'lechones 0-3 semanas' 'lechones 1 semana' 'lechones 28-42 di

In [107]:
# PigAge mapping
PigStageMapping = {

    'cerdas': 'Sow',
'cebo': 'Finishing',
'lechones 3 meses': 'Finishing',
'4 meses': 'Finishing',
'-': 'Missing',
'lechones': 'Suckling',
'50 dias': 'Nursery',
'1 mes': 'Weaning',
'1-9 semanas': 'Mixed',
'6 semanas': 'Nursery',
'10 semanas': 'Nursery',
'nuliparas': 'Gilt',
'recria': 'Gilt',
'45 dias': 'Nursery',
'3 meses': 'Finishing',
'reposicion': 'Gilt',
'destete': 'Weaning',
'transicion y cebo': 'Mixed',
'primerizas': 'Sow',
'cebo 100kg': 'Finishing',
'6 meses': 'Finishing',
'transicion': 'Nursery',
'9 semanas': 'Nursery',
'7 dias': 'Suckling',
'cebo 130 dias': 'Finishing',
'cerdas gestacion': 'Sow',
'8 semanas': 'Nursery',
'80 dias': 'Finishing',
'180 dias': 'Finishing',
'10 meses': 'Finishing',
'12 semanas': 'Finishing',
'100 kg': 'Finishing',
'final cebo': 'Finishing',
'precebo': 'Nursery',
'90 dias': 'Finishing',
'140 dias': 'Finishing',
'3-9 semanas': 'Nursery',
'100 dias': 'Finishing',
'primales': 'Finishing',
'5 meses': 'Finishing',
'110 dias': 'Finishing',
'18 dias': 'Suckling',
'35 kg': 'Finishing',
'2 meses': 'Nursery',
'3,5 meses': 'Finishing',
'5 semanas': 'Nursery',
'50-80 dias': 'Nursery',
'cebo 5 meses': 'Finishing',
'7 semanas': 'Nursery',
'cerdas recria': 'Gilt',
'17 semanas': 'Finishing',
'8-9 meses': 'Finishing',
'cebo 70 kg': 'Finishing',
'26 semanas': 'Finishing',
'3 semanas': 'Nursery',
'14 semanas': 'Finishing',
'25 semanas': 'Finishing',
'60 dias': 'Nursery',
'lactacion': 'Suckling',
'75 dias': 'Finishing',
'2,5 meses': 'Finishing',
'verraco': 'Boar',
'35 dias': 'Nursery',
'30 dias': 'Nursery',
'16 semanas': 'Finishing',
'40 dias': 'Finishing',
'70 dias': 'Finishing',
'cebo 30 kg': 'Finishing',
'cerdas 10 meses': 'Finishing',
'post-destete': 'Nursery',
'9 meses': 'Finishing',
'cerdas reposicion': 'Gilt',
'28 semanas': 'Finishing',
'8 meses': 'Finishing',
'18 semanas': 'Finishing',
'120 dias': 'Finishing',
'cebo 120 dias': 'Finishing',
'49 dias': 'Nursery',
'11 semanas': 'Finishing',
'primalas': 'Gilt',
'24 semanas': 'Finishing',
'21 dias': 'Weaning',
'1-3 anos': 'Other',
'cebo 55 kg': 'Finishing',
'3 dias': 'Suckling',
'25 kg': 'Finishing',
'cebo 14 semanas': 'Finishing',
'90 kg': 'Finishing',
'80 kg': 'Finishing',
'54 dias': 'Nursery',
'30 kg': 'Finishing',
'20 kg': 'Finishing',
'160 dias': 'Finishing',
'14-16 semanas': 'Finishing',
'precebo 15 kg': 'Nursery',
'19 semanas': 'Finishing',
'4 semanas': 'Weaning',
'33 dias': 'Nursery',
'gestacion': 'Sow',
'13 semanas': 'Finishing',
'3-4 meses': 'Finishing',
'5-7 semanas': 'Nursery',
'50 kg': 'Finishing',
'4-9 semanas': 'Nursery',
'3-4 semanas': 'Weaning',
'ceno': 'Other',
'25-30 kg': 'Finishing',
'entrada cebo': 'Nursery',
'15 semanas': 'Finishing',
'10-13 semanas': 'Finishing',
'20 dias': 'Suckling',
'cebo 90 kg': 'Finishing',
'cebo 80 kg': 'Finishing',
'cerdas lactacion': 'Sow',
'cebo 40 kg': 'Finishing',
'85 dias': 'Finishing',
'final transicion': 'Mixed',
'20 semanas': 'Finishing',
'22 semanas': 'Finishing',
'2 a 7 meses': 'Finishing',
'3 a 5 meses': 'Finishing',
'42 dias': 'Nursery',
'2 anos': 'Other',
'150 dias': 'Finishing',
'futuras reproductoras': 'Gilt',
'cerdas nuliparas': 'Gilt',
'71 dias': 'Finishing',
'25 dias': 'Weaning',
'5-10 semanas': 'Nursery',
'4-10 semanas': 'Nursery',
'5-9 semanas': 'Nursery',
'4-8 semanas': 'Nursery',
'5-8 semanas': 'Nursery',
'240 dias': 'Finishing',
'2-3 semanas': 'Mixed',
'6-7 semanas': 'Nursery',
'1-2 semanas post-destete': 'Nursery',
'6-8 semanas': 'Nursery',
'lechones transicion': 'Nursery',
'reproductoras': 'Sow',
'lechones destete': 'Weaning',
'lechonera': 'Suckling',
'lechones 4 semanas': 'Weaning',
'lechones 5-6 semanas': 'Nursery',
'lechones 15 dias': 'Suckling',
'7 meses': 'Finishing',
'lechones 3 semanas': 'Weaning',
'lechones 6 semanas': 'Nursery',
'lechones 54 dias': 'Nursery',
'nacidos muertos': 'Sow',
'lechones 10 semanas': 'Nursery',
'lechones 56 dias': 'Nursery',
'multiparas': 'Sow',
'lechones 1-8 semanas': 'Mixed',
'lechones lactacion': 'Suckling',
'lechones 0-3 semanas': 'Suckling',
'lechones 1 semana': 'Suckling',
'lechones 28-42 dias': 'Nursery',
'lechones paridera': 'Suckling',
'lechones 3-9 semanas': 'Mixed',
'verracos': 'Boar',
'10-22 semanas': 'Finishing',
'lechones 5 semanas': 'Nursery',
'lechones 1-4 semanas': 'Suckling',
'lechones 1-2 semanas': 'Suckling',
'lechones 2 semanas': 'Suckling',
'abortos': 'Sow',
'6-9 semanas': 'Nursery',
'4-7 semanas': 'Nursery',
'lechones 1-3 semanas': 'Suckling',
'lechones 2 dias': 'Suckling',
'lechones 38 dias': 'Nursery',
'37-58 dias': 'Nursery',
'2 semanas': 'Suckling',
'lechones 21 dias': 'Weaning',
'2-3 meses': 'Finishing',
'1-4 semanas': 'Suckling',
'7-8 semanas': 'Nursery',
'26 dias': 'Weaning',
'24 dias': 'Weaning',

  
}

# Map the 'Edad' column using the English translation mapping
df1['PigStage'] = df1['Age'].map(PigStageMapping).fillna("Missing")
df1 = df1.drop(['Age'], axis=1)


In [108]:
df1

,SampleNumber,Date,Province,Pathogen,Result,SampleType,DiagnosticTest,Country,Breed,Lab_Reference,PigStage
0,67337/2024,2024-01-10,Segovia,Swine influenza virus,0,Tissue,PCR,Spain,Meat,6,Missing
1,67360/2024,2024-01-11,Segovia,Swine influenza virus,0,Tissue,PCR,Spain,Meat,6,Missing
2,67440/2024,2024-01-17,Ourense,Swine influenza virus,0,Tissue,PCR,Spain,Meat,6,Nursery
3,67454/2024,2024-01-18,Badajoz,Swine influenza virus,0,Tissue,PCR,Spain,Meat,6,Nursery
4,67515/2024,2024-01-19,Murcia,Swine influenza virus,0,Fluid,PCR,Spain,Meat,6,Nursery
...,...,...,...,...,...,...,...,...,...,...,...
3050,70755/2024,2024-09-12,Burgos,Porcine epidemic diarrhea virus,0,Swab,PCR,Spain,Meat,6,Finishing
3051,70996/2024,2024-10-01,Segovia,Porcine epidemic diarrhea virus,0,Faeces,PCR,Spain,Meat,6,Finishing
3052,71012/2024,2024-10-02,Segovia,Porcine epidemic diarrhea virus,0,Faeces,PCR,Spain,Meat,6,Finishing
3053,71267/2024,2024-10-18,Ourense,Porcine epidemic diarrhea virus,0,Faeces,PCR,Spain,Meat,6,Missing


In [109]:
# Save to desired path
df1.to_csv('output/PigLab6_2.csv', sep= ';', index=False)

In [110]:
   barometer = df

In [111]:
barometer

,SampleNumber,Province,Date,Pathogen,Result,SampleType,DiagnosticTest,Country,Breed,Lab_Reference,PigStage
0,50948/2021,Segovia,2021-01-08,Swine influenza virus,0,Tissue,PCR,Spain,Meat,6,Sow
1,50974/2021,Segovia,2021-01-14,Lawsonia intracellularis,0,Tissue,PCR,Spain,Meat,6,Finishing
2,50978/2021,Avila,2021-01-14,Lawsonia intracellularis,0,Faeces,PCR,Spain,Meat,6,Finishing
3,51015/2021,Segovia,2021-01-15,Lawsonia intracellularis,0,Faeces,PCR,Spain,Meat,6,Finishing
4,51061/2021,Murcia,2021-01-19,Swine influenza virus,0,Tissue,PCR,Spain,Meat,6,Finishing
...,...,...,...,...,...,...,...,...,...,...,...
1231,67162/2023,Segovia,2023-12-21,Lawsonia intracellularis,1,Faeces,PCR,Spain,Meat,6,Finishing
1232,67187/2023,Segovia,2023-12-22,Lawsonia intracellularis,1,Swab,PCR,Spain,Meat,6,Finishing
1233,67205/2023,Segovia,2023-12-27,Lawsonia intracellularis,0,Faeces,PCR,Spain,Meat,6,Missing
1234,67209/2023,Segovia,2023-12-27,Swine influenza virus,0,Tissue,PCR,Spain,Meat,6,Weaning


## Join both databases

In [112]:
df_all = pd.concat([df, df1], ignore_index = True)

print(df_all['Pathogen'].unique())

['Swine influenza virus' 'Lawsonia intracellularis'
 'Mycoplasma hyopneumoniae' 'Actinobacillus pleuropneumoniae'
 'Porcine reproductive and respiratory syndrome virus'
 'Brachyspira hyodysenteriae' 'Porcine epidemic diarrhea virus']


In [113]:
print(df_all['PigStage'].unique())

['Sow' 'Finishing' 'Missing' 'Suckling' 'Nursery' 'Weaning' 'Mixed' 'Gilt'
 'Boar' 'Other']


In [114]:
print(df_all['SampleType'].unique())

['Tissue' 'Faeces' 'Fluid' 'Swab' 'Other' 'Missing' 'BAL' 'Blood' 'Mixed']


In [115]:
df_all.to_csv('output/PigLab6_update20250109.csv', sep=';', index=False)

In [116]:
df_all

,SampleNumber,Province,Date,Pathogen,Result,SampleType,DiagnosticTest,Country,Breed,Lab_Reference,PigStage
0,50948/2021,Segovia,2021-01-08,Swine influenza virus,0,Tissue,PCR,Spain,Meat,6,Sow
1,50974/2021,Segovia,2021-01-14,Lawsonia intracellularis,0,Tissue,PCR,Spain,Meat,6,Finishing
2,50978/2021,Avila,2021-01-14,Lawsonia intracellularis,0,Faeces,PCR,Spain,Meat,6,Finishing
3,51015/2021,Segovia,2021-01-15,Lawsonia intracellularis,0,Faeces,PCR,Spain,Meat,6,Finishing
4,51061/2021,Murcia,2021-01-19,Swine influenza virus,0,Tissue,PCR,Spain,Meat,6,Finishing
...,...,...,...,...,...,...,...,...,...,...,...
4286,70755/2024,Burgos,2024-09-12,Porcine epidemic diarrhea virus,0,Swab,PCR,Spain,Meat,6,Finishing
4287,70996/2024,Segovia,2024-10-01,Porcine epidemic diarrhea virus,0,Faeces,PCR,Spain,Meat,6,Finishing
4288,71012/2024,Segovia,2024-10-02,Porcine epidemic diarrhea virus,0,Faeces,PCR,Spain,Meat,6,Finishing
4289,71267/2024,Ourense,2024-10-18,Porcine epidemic diarrhea virus,0,Faeces,PCR,Spain,Meat,6,Missing


### Step 02: Create an RDF graph and namespaces.

In [117]:
g = rdflib.Graph()
xsd = Namespace('http://www.w3.org/2001/XMLSchema#')
g.bind('xsd', xsd)

# Define your custom namespace for your ontology's properties
LHO = Namespace("http://www.purl.org/decide/LiveStockHealthOnto/LHO#")
g.bind('LHO', LHO)

SKOS = Namespace("http://www.w3.org/2004/02/skos/core#")
g.bind('skos', SKOS)

decide=  Namespace("http://www.purl.org/decide#")
g.bind('decide', decide)
ncit = Namespace("http://purl.obolibrary.org/obo/NCIT_C25464")
g.bind('ncit', ncit)
agrovoc = Namespace ("http://aims.fao.org/aos/agrovoc")
g.bind('agrovoc', agrovoc)

###  Step 03: Iterate over the Panda DataFrame and map to ontology properties:

In [118]:
 # for slugging local names or avoid spaces in names to create URIs along with label names

import re
import urllib.parse

def _safe_local_name(v):
    v = str(v).strip()
    # RFC3986 unreserved: A-Z a-z 0-9 - . _ ~
    v = re.sub(r'[^A-Za-z0-9\-._~]', '_', v)
    v = re.sub(r'_+', '_', v)  # collapse repeated underscores (optional)
    return v

In [119]:
#For Pig Data frame RDF conversion
#for index, row in barometer.head(500).iterrows():
##................................><.........................................................................
##Note: spaces in URIs are not OK. RDFLib (and the RDF specs) require valid URIs; a space makes them invalid. 
##If there is space in names then got errors in the Turtle serialization.
##................................><.........................................................................
for index, row in df_all.iterrows():
    SampleNumber = row["SampleNumber"]
    Date = row["Date"]
    Pathogen = row["Pathogen"]
    Breed = row["Breed"]
    Country = row["Country"]
    Province = row["Province"]
    SampleType = row["SampleType"]
    SampleResult = row["Result"]
    DiagnosticTest = row["DiagnosticTest"]
    PigStage= row["PigStage"]

    
    # Create a unique URI for each sample based on the row index
    PigSample_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#Lab1PigSample_{index}")
    
    # Add sample type assertion
    g.add((PigSample_uri, RDF.type, LHO.PigSample))
    description = "An individual representing a sample from a Pig and Piglets."
    g.add((PigSample_uri, RDFS.comment, Literal(description, lang="en")))

    
    #if "Pathogen" in row and row["Pathogen"]:    # Making little changes to pathogen URI so that it will not create error
        #Pathogen = row["Pathogen"]
        #Pathogen_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Pathogen}")
        #g.add((PigSample_uri, LHO.hasPathogen, Pathogen_uri))
        #g.add((Pathogen_uri, RDF.type, decide.Pathogen))
        #description = "An individual representing Pig pathogen."
        #g.add((Pathogen_uri, RDFS.comment, Literal(description, lang="en")))

    if "Pathogen" in row and row["Pathogen"]:
        Pathogen = str(row["Pathogen"]).strip()
        Pathogen_local = _safe_local_name(Pathogen)  # e.g., "Swine_influenza_virus"
        Pathogen_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Pathogen_local}")
        g.add((PigSample_uri, LHO.hasPathogen, Pathogen_uri))
        g.add((Pathogen_uri, RDF.type, decide.Pathogen))
        description = "An individual representing Pig pathogen."
        g.add((Pathogen_uri, RDFS.comment, Literal(description, lang="en")))
        g.add((Pathogen_uri, RDFS.label, Literal(Pathogen)))  # human-readable conventional label, e.g., "Swine influenza virus" same thing we can do with other properties like breed and diagnostic test etc. if we have sapce

        
    if "Breed" in row and row["Breed"]:
        Breed = row["Breed"]
        Breed_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Breed}")
        g.add((PigSample_uri, LHO.hasBreed, Breed_uri))
        g.add((Breed_uri, RDF.type, LHO.Breed))
        description = "An individual representing a Piglet breed which is Meat."
        g.add((Breed_uri, RDFS.comment, Literal(description, lang="en")))

    if "Lab_Reference" in row and row["Lab_Reference"]:  #Adding that code
        Lab_Reference = row["Lab_Reference"]
        Lab_Reference_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Lab_Reference}")
        g.add((PigSample_uri, LHO.hasLab_Reference, Lab_Reference_uri))
        g.add((Lab_Reference_uri, RDF.type, LHO.Lab_Reference))
        description = "An individual representing from which lab refence it belongs."
        g.add((Lab_Reference_uri, RDFS.comment, Literal(description, lang="en")))
        
    if "DiagnosticTest" in row and row["DiagnosticTest"]:
        DiagnosticTest= row["DiagnosticTest"]
        DiagnosticTest_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{DiagnosticTest}")
        g.add((PigSample_uri, LHO.hasDiagnosticTest, DiagnosticTest_uri))
        g.add((DiagnosticTest_uri, RDF.type, LHO.DiagnosticTest))
    if DiagnosticTest == "PCR":
        description = "An individual representing DNA/RNA amplification for rapid pathogen detection."
    elif SampleType == "Culture":
        description = "An individual representing a Growing live microorganisms for identification."
        g.add((DiagnosticTest_uri, RDFS.comment, Literal(description, lang="en")))
        
    if "Country" in row and row["Country"]:
        Country = row["Country"].strip()  # Use strip() to remove leading and trailing spaces
        # Encode the Country value to create a valid URI
        Country = urllib.parse.quote(Country)
        Country_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Country}")
    
        # Map to the specific class URI in your ontology and add the label
        g.add((PigSample_uri, LHO.hasCountry, Country_uri))
        g.add((Country_uri, RDF.type, URIRef("http://purl.obolibrary.org/obo/NCIT_C25464")))  # Use the specific class URI
        g.add((Country_uri, RDFS.label, Literal("Spain")))
        description = "An individual representing different Countries."
        g.add((Country_uri, RDFS.comment, Literal(description, lang="en")))
        
    if "Province" in row and row["Province"]:
        Province = row["Province"].strip()
        Province = urllib.parse.quote(Province)
        Province_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Province}")
        g.add((PigSample_uri, LHO.hasProvince, Province_uri))
        g.add((Province_uri, RDF.type, LHO.Province))
        description = "An individual representing different Province."
        g.add((Province_uri, RDFS.comment, Literal(description, lang="en")))
    
    if "PigStage" in row and row["PigStage"]:
        Stage = row["PigStage"]
        Stage_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Stage}")
        g.add((PigSample_uri, LHO.hasProductionStages, Stage_uri))
        g.add((Stage_uri, RDF.type, LHO.PigProductionStage))
        description = "An individual representing Production stage ."
        g.add((Stage_uri, RDFS.comment, Literal(description, lang="en")))
    
    if "SampleType" in row and row["SampleType"]:
        SampleType = row["SampleType"]
        SampleType_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{SampleType}")
        # Add statements for SampleType
        g.add((PigSample_uri, LHO.hasSampleType, SampleType_uri))
        g.add((SampleType_uri, RDF.type, URIRef("http://www.purl.org/decide#SampleType")))
    if SampleType == "Autopsy":
        description = "An individual representing a cattle sample obtained through autopsy."
    elif SampleType == "BAL":
        description = "An individual representing a cattle sample obtained through bronchoalveolar lavage (BAL)."
    elif SampleType == "SWAB":
        description = "An individual representing a cattle sample obtained through swabbing."
    elif SampleType == "Missing":
        description = "An individual representing a missing or unspecified cattle sample type."
        g.add((SampleType_uri, RDFS.comment, Literal(description, lang="en")))
           
    if "Result" in row and row["Result"]:
        SampleResult = row["Result"]
        # Create a unique URI for the Result based on the value
        Result_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{SampleResult}")
        g.add((PigSample_uri, LHO.hasResult, Result_uri))
        g.add((Result_uri, RDF.type, decide.SampleResult))
    if SampleResult == "1":
        description = "An individual representing a Positive test result"
    elif SampleResult == "0":
        description = "An individual representing a negative test result"
    elif SampleResult == "missing":
        description = "An individual representing an unknown or missing test result."
    g.add((Result_uri, RDFS.comment, Literal(description, lang="en")))


    if "Date" in row and not pd.isnull(row["Date"]):  # Check for NaN or None values
        Date = str(row["Date"]).strip()
    
    # Split the datetime string and take the date part
        Date = Date.split()[0]
    
    # Create a unique URI for the Date based on the value (use the Date directly)
        Date_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Date}")
    
    # Add Date assertion
        g.add((PigSample_uri, LHO.hasDate, Date_uri))
        g.add((Date_uri, RDF.type, LHO.Date))
    

# Serialize the RDF graph to a file
rdf_output_file = "output/PigLab1.rdf"
g.serialize(rdf_output_file, format="xml")

# Serialize the RDF graph to Turtle format and print it too see RDF conversion
#turtle_data = g.serialize(format="turtle")
#print(turtle_data)

<Graph identifier=Ne966ff11f4614832b2e68115112601e8 (<class 'rdflib.graph.Graph'>)>

### Step 4: For Knowledge Graph,Load the RDF data and ontology into a Panda DataFrame: 

In [120]:
# Parse the ontology file in OWL format and add it to the graph
path_to_ontology = "Ontology/LivestockHealthOntology.owl"
g.parse(path_to_ontology, format="xml")

<Graph identifier=Ne966ff11f4614832b2e68115112601e8 (<class 'rdflib.graph.Graph'>)>

### Step 05: Query the data from updated ontology 

In [121]:
query = """
PREFIX rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX decide: <http://www.purl.org/decide#>
PREFIX rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX LHO:   <http://www.purl.org/decide/LiveStockHealthOnto/LHO#>
PREFIX skos:  <http://www.w3.org/2004/02/skos/core#>
PREFIX xsd:   <http://www.w3.org/2001/XMLSchema#>

SELECT
  (STRAFTER(STR(?Sample), "#") AS ?SampleNumber)
  (STRAFTER(STR(?Province), "#") AS ?Province)
  (?FlooredDate AS ?FlooredDate)
  (COALESCE(STR(?PathogenLabel), REPLACE(STRAFTER(STR(?Pathogen), "#"), "_", " ")) AS ?Pathogen)
  (STRAFTER(STR(?SampleResult), "#") AS ?Result)
  (STRAFTER(STR(?SampleType), "#") AS ?SampleType)
  (STRAFTER(STR(?DiagnosticTest), "#") AS ?DiagnosticTest)
  (STRAFTER(STR(?Country), "#") AS ?Country)
  (STRAFTER(STR(?Breed), "#") AS ?Breed)
  (STRAFTER(STR(?Lab_Reference), "#") AS ?Lab_Reference)
  (STRAFTER(STR(?PigProductionStages), "#") AS ?PigStage)
  (STRAFTER(STR(?AutonomousCommunity), "#") AS ?AutonomousCommunity)
  (?MonthNum AS ?Month)
  (?YearNum  AS ?Year)
WHERE {
  ?Sample rdf:type LHO:PigSample .
  ?Sample LHO:hasDate ?Date .
  ?Sample LHO:hasPathogen ?Pathogen .
  ?Sample LHO:hasBreed ?Breed .
  ?Sample LHO:hasSampleType ?SampleType .
  ?Sample LHO:hasDiagnosticTest ?DiagnosticTest .
  ?Sample LHO:hasProductionStages ?PigProductionStages .
  ?Sample LHO:hasProvince ?Province .
  ?Sample LHO:hasCountry ?Country .
  ?Sample LHO:hasResult ?SampleResult .
  ?Sample LHO:hasLab_Reference ?Lab_Reference .
  OPTIONAL { ?Sample LHO:hasAutonomousCommunity ?AutonomousCommunity . }

  # Pathogen label only (prefer rdfs:label/skos:prefLabel; prefer en or no-lang)
  OPTIONAL {
    ?Pathogen (rdfs:label|skos:prefLabel) ?PathogenLabel .
    FILTER(LANG(?PathogenLabel) = "" || LANGMATCHES(LANG(?PathogenLabel), "en"))
  }

  # ---- Date parsing (supports DD/MM/YYYY and YYYY-MM-DD) ----
  BIND( STRAFTER(STR(?Date), "#") AS ?DateStr )
  BIND( ?DateStr AS ?FlooredDate )

  # month
  BIND(
    xsd:integer(
      IF(CONTAINS(?DateStr, "/"),
         SUBSTR(?DateStr, 4, 2),   # DD/MM/YYYY -> MM is positions 4-5
         SUBSTR(?DateStr, 6, 2)    # YYYY-MM-DD -> MM is positions 6-7
      )
    ) AS ?MonthNum
  )

  # year
  BIND(
    xsd:integer(
      IF(CONTAINS(?DateStr, "/"),
         SUBSTR(?DateStr, 7, 4),   # DD/MM/YYYY -> YYYY starts at 7
         SUBSTR(?DateStr, 1, 4)    # YYYY-MM-DD -> YYYY is positions 1-4
      )
    ) AS ?YearNum
  )
}
"""
results = g.query(query)

data = []
for row in results:
    data.append(list(row))

df = pd.DataFrame(data, columns=[
    "SampleNumber","Province","FlooredDate","Pathogen","Result","SampleType",
    "DiagnosticTest","Country","Breed","LabReference","PigStage",
    "AutonomousCommunity","Month","Year"
])

# Display the dataframe
df

,SampleNumber,Province,FlooredDate,Pathogen,Result,SampleType,DiagnosticTest,Country,Breed,LabReference,PigStage,AutonomousCommunity,Month,Year
0,Lab1PigSample_0,Segovia,2021-01-08,Swine influenza virus,0,Tissue,PCR,Spain,Meat,6,Sow,None,1,2021
1,Lab1PigSample_1,Segovia,2021-01-14,Lawsonia intracellularis,0,Tissue,PCR,Spain,Meat,6,Finishing,None,1,2021
2,Lab1PigSample_2,Avila,2021-01-14,Lawsonia intracellularis,0,Faeces,PCR,Spain,Meat,6,Finishing,None,1,2021
3,Lab1PigSample_3,Segovia,2021-01-15,Lawsonia intracellularis,0,Faeces,PCR,Spain,Meat,6,Finishing,None,1,2021
4,Lab1PigSample_4,Murcia,2021-01-19,Swine influenza virus,0,Tissue,PCR,Spain,Meat,6,Finishing,None,1,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4278,Lab1PigSample_4286,Burgos,2024-09-12,Porcine epidemic diarrhea virus,0,Swab,PCR,Spain,Meat,6,Finishing,None,9,2024
4279,Lab1PigSample_4287,Segovia,2024-10-01,Porcine epidemic diarrhea virus,0,Faeces,PCR,Spain,Meat,6,Finishing,None,10,2024
4280,Lab1PigSample_4288,Segovia,2024-10-02,Porcine epidemic diarrhea virus,0,Faeces,PCR,Spain,Meat,6,Finishing,None,10,2024
4281,Lab1PigSample_4289,Ourense,2024-10-18,Porcine epidemic diarrhea virus,0,Faeces,PCR,Spain,Meat,6,Missing,None,10,2024


In [122]:
# Save Results: save to CSV
import os
os.makedirs("output", exist_ok=True)

df.to_csv("output/PigLab6_query_results.csv", index=False, encoding="utf-8")
print("Saved: output/PigLab6_query_results_baromterepig.csv")

Saved: output/PigLab6_query_results_baromterepig.csv
